In [2]:
#!pip install tensorflow-gpu keras
import os
import sys

import pandas as pd
import numpy as np
import datetime as dt

# import plotly.offline as ofply
# import plotly.graph_objs as go
# from plotly import tools as plytools

# %matplotlib inline
# # import matplotlib
# import matplotlib.pyplot as plt

import importlib
from IPython.display import Markdown
import IPython
# ofply.init_notebook_mode()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.core.config.options.display.max_columns = 100
import keras
from keras.layers import Input, Dense, LSTM, Bidirectional, Flatten, LocallyConnected1D, BatchNormalization, Conv1D, Conv2D, Conv3D, Reshape, Activation, Add, RepeatVector, MaxoutDense
from keras.layers import MaxPool1D, AvgPool1D, concatenate, ActivityRegularization, GlobalAvgPool1D, Dropout, Lambda, UpSampling1D, TimeDistributed, GlobalAvgPool2D, AlphaDropout
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
#from keras.constraints import maxnorm
#from keras.wrappers.scikit_learn import KerasRegressor
from keras.backend import clear_session
from keras import layers
from keras import optimizers
from keras import initializers
from keras import regularizers
from keras import activations
from keras import backend as K
from keras.layers.merge import add
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_validate, ParameterGrid, KFold, StratifiedKFold
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import pandas
import pandas as pd
from keras.layers.merge import Concatenate
from keras.layers.convolutional import Conv2D,MaxPooling2D,ZeroPadding2D,AveragePooling2D,Conv1D,MaxPooling1D





Using TensorFlow backend.


In [4]:
# load dev dataset dictionary
dev_dataset= pd.read_pickle('Z:/eee_comms1/Shared/Avec2018Project/pkls/auc_pkls/dev_tw_dataset_mat_dict.pkl')
aur_dev_tw = dev_dataset['au_r_dataset_mat']
auc_dev_tw= dev_dataset['au_c_dataset_mat']
eye_dev_tw = dev_dataset['eye_dataset_mat']
pose_dev_tw = dev_dataset['pose_dataset_mat']
Y_int= dev_dataset['Y']


Ymrs_dev= dev_dataset['Ymrs'] / max(dev_dataset['Ymrs'])   # maybe standardization is better
del dev_dataset

# confirm shapes
print('aur_dev',aur_dev_tw.shape)
print('auc_dev',auc_dev_tw.shape)
print('eye_dev',eye_dev_tw.shape)
print('Ymrs_dev',Ymrs_dev.shape)
print('Y_train',Y_int.shape)

# One-hot encode the labels
y_encoder = OneHotEncoder()
y_sparse = y_encoder.fit_transform(Y_int.reshape((len(Y_int),1)))
Y_oh_dev = y_sparse.toarray()
print('Y_oh.shape',Y_oh_dev.shape)


aur_dev (230, 1560, 17)
auc_dev (230, 1560, 18)
eye_dev (230, 1560, 8)
Ymrs_dev (230,)
Y_train (230,)
Y_oh.shape (230, 3)


C:\Users\sarmad\Anaconda3\envs\keras_v\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
# load train dataset dictionary
train_dataset= pd.read_pickle('Z:/eee_comms1/Shared/Avec2018Project/pkls/auc_pkls/train_tw_dataset_mat_dict.pkl')
aur_train_tw = train_dataset['au_r_dataset_mat']
auc_train_tw = train_dataset['au_c_dataset_mat']
eye_train_tw= train_dataset['eye_dataset_mat']
pose_train_tw = train_dataset['pose_dataset_mat']
Y_int= train_dataset['Y']


Ymrs_train= train_dataset['Ymrs'] / max(train_dataset['Ymrs'])   # maybe standardization is better
del train_dataset

# confirm shapes
print('aur_train',aur_train_tw.shape)
print('auc_train',auc_train_tw.shape)
print('eye_train',eye_train_tw.shape)
print('Ymrs_train',Ymrs_train.shape)
print('Y_train',Y_int.shape)

# One-hot encode the labels
y_encoder = OneHotEncoder()
y_sparse = y_encoder.fit_transform(Y_int.reshape((len(Y_int),1)))
Y_oh_tr = y_sparse.toarray()
print('Y_oh.shape',Y_oh_tr.shape)


In [5]:
# Selected summerised features



# load dev dataset dictionary 
eye_dataset= pd.read_pickle('Z:/eee_comms1/Shared/Avec2018Project/pkls/auc_pkls/eye_classification.pkl')
eye_train=eye_dataset['sel_train_bagging_train']
eye_dev=eye_dataset['sel_test_bagging_dev']


print('eye_train',eye_train.shape)
print('eye_dev',eye_dev.shape)

print('**************')
print('              ')

pose_dataset= pd.read_pickle('Z:/eee_comms1/Shared/Avec2018Project/pkls/auc_pkls/pose_classification.pkl')
pose_train=pose_dataset['sel_train_bagging_train']
pose_dev=pose_dataset['sel_test_bagging_dev']

print('pose_train',pose_train.shape)
print('pose_dev',pose_dev.shape)


print('**************')
print('              ')


auc_dataset= pd.read_pickle('Z:/eee_comms1/Shared/Avec2018Project/pkls/auc_pkls/auc_classification.pkl')
auc_train=auc_dataset['sel_train_bagging_train']
auc_dev=auc_dataset['sel_test_bagging_dev']


print('auc_train',auc_train.shape)
print('auc_dev',auc_dev.shape)



print('**************')
print('              ')


sumerized_train_feats=pd.concat([auc_train,eye_train,pose_train], axis=1, sort=False)
sumerized_dev_feats=pd.concat([auc_dev,eye_dev,pose_dev], axis=1, sort=False)



print('sumerized_train_feats',sumerized_train_feats.shape)
print('sumerized_dev_feats',sumerized_dev_feats.shape)

eye_train (104, 56)
eye_dev (60, 56)
**************
              
pose_train (104, 20)
pose_dev (60, 20)
**************
              
auc_train (104, 78)
auc_dev (60, 78)
**************
              
sumerized_train_feats (104, 154)
sumerized_dev_feats (60, 154)


In [0]:
def GlobalRichPool(inputs, name=None):

    out_avg = Lambda(K.mean, arguments={'axis':1})(inputs)
    out_std = Lambda(K.std, arguments={'axis':1})(inputs)
    out_max = Lambda(K.max, arguments={'axis':1})(inputs)
#     out_lse = Lambda(K.logsumexp, arguments={'axis':1})(inputs)
    
    if name:
        out = concatenate([out_avg, out_std, out_max], name=name)
    else:
        out = concatenate([out_avg, out_std, out_max])
    
    return out
  
  
def bnrelu(inpt):
    inpt = BatchNormalization(axis=-1)(inpt)
    inpt = layers.ReLU()(inpt)
#     inpt = Activation('relu')(inpt)
    return inpt

In [0]:
# Joint sequence extraction
def conv_seq():

    inshape_aur = aur_train_tw.shape[1:]
    inshape_auc = auc_train_tw.shape[1:]
    inshape_eye = eye_train_tw.shape[1:]
    inshape_pose = pose_train_tw.shape[1:]
    inshape_ofsum = sumerized_train_feats.shape[1:]
    
    outshape = len(np.unique(Y_int))

    ## some defaults
    # initializers, optimizers, constraints
    adam = optimizers.adam()
    init = initializers.he_normal()
    fc_act = 'relu'
    init_c = initializers.he_uniform()
    binit = 'zeros'
    #    binit_c = initializers.glorot_normal()
    binit_c = 'zeros'

    outlist = []
    outlist2 = []

    ### Model declaration
    ## Inputs
    # AU
    aur_in = Input(shape=inshape_aur, name='AUR_input')
    print('aur_in shape',aur_in)
    aur = aur_in
    raw_globalpool = GlobalRichPool(aur, name='raw_globalpool_aur')
    outlist.append(raw_globalpool)

    auc_in = Input(shape=inshape_auc, name='AUC_input')
    auc = auc_in
    raw_globalpool = GlobalRichPool(auc, name='raw_globalpool_auc')
    outlist.append(raw_globalpool)

    print('auc in ',auc.shape)
    # EYE
    eye_in = Input(shape=inshape_eye, name='EYE_input')
    eye = eye_in
    raw_globalpool = GlobalRichPool(eye, name='raw_globalpool_eye')
    outlist.append(raw_globalpool)

    # POSE
    pose_in = Input(shape=inshape_pose, name='Pose_input')
    pose = pose_in
    raw_globalpool = GlobalRichPool(pose, name='raw_globalpool_pose')
    outlist.append(raw_globalpool)
    
    # Openface classification summary
    ofsum_in = Input(shape=inshape_ofsum, name='OFsum_input')
    ofsum = ofsum_in
    outlist.append(ofsum)
    
    
    
    # Concat over channels and
    # sequence extraction
    combined = concatenate([aur,auc, eye, pose], name='chan_concat')
    print('combined',combined.shape)

    

   
    kreg = regularizers.l1_l2(l1=0.1)
    
    combined = BatchNormalization()(combined)

    seq = Conv1D(filters=64, kernel_size=5, strides=1, padding='same', name='seq1')(combined)
    seq=bnrelu(seq)
    seq = MaxPooling1D(pool_size=2, strides=2)(seq)
    
    
    
    seq = Conv1D(filters=96, kernel_size=5, strides=1,padding='same', name='seq2')(seq)
    seq=bnrelu(seq)
    seq =MaxPooling1D(pool_size=2, strides=2)(seq)
    
    
    
    seq = Conv1D(filters=96, kernel_size=5, strides=1, padding='same', name='seq3')(seq)
    seq=bnrelu(seq)
    globalpool = GlobalRichPool(seq, name='seq_globalpool_ms')
    outlist.append(globalpool)
    # receptive field = 20 timesteps
    
    
    
    
    seq = MaxPooling1D(pool_size=2, strides=2)(seq)
    seq = Conv1D(filters=96, kernel_size=5, strides=1,padding='same', name='seq4')(seq)
    seq=bnrelu(seq)
    seq = MaxPooling1D(pool_size=2, strides=2)(seq)
    
    
    seq = Conv1D(filters=96, kernel_size=5, strides=1,padding='same', name='seq5')(seq)
    seq=bnrelu(seq)
    globalpool = GlobalRichPool(seq, name='seq_globalpool_s')
    outlist.append(globalpool)
    # receptive field = 80 timesteps
    
    
    
    seq = MaxPooling1D(pool_size=3, strides=3)(seq)
    seq = Conv1D(filters=96, kernel_size=5, strides=1, padding='same',name='seq6')(seq)
    seq=bnrelu(seq)
    globalpool = GlobalRichPool(seq, name='seq_globalpool_8s')
    
    
    
    outlist.append(globalpool)
    # receptive field = 240 timesteps, 8 seconds
    
    # Fully connected layers

    out_combined = concatenate(outlist, name='feat_concat',axis=1)
    out = out_combined
    print('Feat combined shape', out.shape )

    out = BatchNormalization()(out)
    out = Dropout(0.5)(out)
    kreg = regularizers.l1_l2(l1=0, l2=0.01)
    out = Dense(512, activation=fc_act, kernel_regularizer=kreg)(out)
    out = Dense(512, activation=fc_act, kernel_regularizer=kreg)(out)
#     out = Dense(512, activation=fc_act, kernel_regularizer=kreg)(out)


    out_class = Dense(outshape, activation='softmax', name='out_class')(out)
    out_reg = Dense(1, activation='linear', name='out_reg')(out)
    # out_class = Lambda(gen_class_output, name='out_class')(out_reg)


    model = Model(inputs=[aur_in,auc_in, eye_in, pose_in, ofsum_in], outputs=[out_class, out_reg])
    metrics = {'out_class':['categorical_accuracy'], 'out_reg':'mean_absolute_error'}
    loss    = {'out_class':'categorical_crossentropy', 'out_reg':'mean_absolute_error'}
    loss_weights = {'out_class': 0.5, 'out_reg':0.5}
    model.compile(optimizer=adam, loss=loss, metrics=metrics, loss_weights=loss_weights)
    
    return model


In [0]:
from sklearn.preprocessing import StandardScaler

def normalize_input(input):
  
  data= input
  scalers = {}
  for i in range(data.shape[2]):
      scalers[i] = StandardScaler()
      data[:, :, i] = scalers[i].fit_transform(data[:, :, i]) 
  return(data)
      
# import numpy as np
# from sklearn.preprocessing import StandardScaler

# train_data=aur_train_tw
# scaler = StandardScaler()
# num_instances, num_time_steps, num_features = train_data.shape

# train_data = np.reshape(train_data, (-1, num_features))
# print('train_data.shape',train_data.shape)
# train_data = scaler.fit_transform(train_data)


# train_data = np.reshape(train_data, (num_instances, num_time_steps, num_features))
# print('train_data',train_data)


from sklearn.preprocessing import StandardScaler

def normalize_2d_input(input):
  
  mean = input.mean(axis=0)
  input -= mean
  std = input.std(axis=0)
  input /= std
 
  return(input)

  
  

In [101]:

# def norm(input):
  
#   mean = input.mean(axis=0)
#   input -= mean
#   std = input.std(axis=0)
#   return input


print('            train data          ')
aur_train_time_w=normalize_input(aur_train_tw)
auc_train_time_w=normalize_input(auc_train_tw)
eye_train_time_w=normalize_input(eye_train_tw)
pose_train_time_w=normalize_input(pose_train_tw)
sumerized_train_feats=normalize_2d_input(sumerized_train_feats)

X_train = [aur_train_time_w, auc_train_time_w, eye_train_time_w, pose_train_time_w]
Y_train = [Y_oh_tr, Ymrs_train]


print('aur_train',aur_train_tw.shape)
print('auc_train',auc_train_tw.shape)
print('eye_train',eye_train_tw.shape)
print('pose_train',pose_train_tw.shape)
print('y_train',Y_train[0].shape)
print('y_train_ymrs',Y_train[1].shape)
print('sumerized_train_feats',sumerized_train_feats.shape)


print('                         ')
print('*************dev data************')
print('                         ')    

aur_dev_time_w=normalize_input(aur_dev_tw)
auc_dev_time_w=normalize_input(auc_dev_tw)
eye_dev_time_w=normalize_input(eye_dev_tw)
pose_dev_time_w=normalize_input(pose_dev_tw)    
sumerized_dev_feats=normalize_2d_input(sumerized_dev_feats)
      
      
X_test = [aur_dev_time_w, auc_dev_time_w, eye_dev_time_w,pose_dev_time_w]
Y_test = [Y_oh_dev, Ymrs_dev]


print('aur_dev',aur_dev_tw.shape)
print('auc_dev',auc_dev_tw.shape)
print('eye_dev',eye_dev_tw.shape)
print('pose_dev',pose_dev_tw.shape)
print('y_dev_',Y_test[0].shape)
print('y_dev_ymrs',Y_test[1].shape)
print('sumerized_dev_feats',sumerized_dev_feats.shape)
print(' pose_dev', pose_dev.shape)


            train data          
aur_train (530, 1560, 17)
auc_train (530, 1560, 18)
eye_train (530, 1560, 8)
pose_train (530, 1560, 6)
y_train (530, 3)
y_train_ymrs (530,)
sumerized_train_feats (104, 154)
                         
*************dev data************
                         
aur_dev (230, 1560, 17)
auc_dev (230, 1560, 18)
eye_dev (230, 1560, 8)
pose_dev (230, 1560, 6)
y_dev_ (230, 3)
y_dev_ymrs (230,)
sumerized_dev_feats (60, 154)
 pose_dev (60, 20)


In [102]:

import matplotlib.pyplot as plt
clear_session()
fitparams = dict(
            shuffle = True,
            batch_size =10,
            epochs=100,
            verbose=1)

model = conv_seq()

model.summary()

# keras.utils.plot_model(model, to_file='conv_seq_plot_model.png', show_shapes=True)
# IPython.display.Image('conv_seq_plot_model.png')



reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.42, patience=12, verbose=1, mode='auto', epsilon=0.0001, cooldown=15, min_lr=1e-5)

# Callbacks




# # reduceLR = ReduceLROnPlateau(monitor='loss', factor=0.42, patience=9, verbose=1, mode='auto', cooldown=10)
# # chkpt_r = ModelCheckpoint(pth_r, monitor='val_reg_mean_absolute_error', mode='min', verbose=2, save_best_only=True, save_weights_only=True, period=1)
# # chkpt_c = ModelCheckpoint(pth_c, monitor='val_clf_acc', mode='max', verbose=2, save_best_only=True, save_weights_only=True, period=1)
# # chkpt_r2c = ModelCheckpoint(pth_r2c, monitor='val_reg2clf_acc', mode='max', verbose=2, save_best_only=True, save_weights_only=True, period=1)




# # Fit
history = model.fit(**fitparams,
            x = X_train,
            y = Y_train,
            validation_data = (X_test, Y_test),
            #validation_split = 0.10,
            callbacks=[reduceLR]
                   )





aur_in shape Tensor("AUR_input:0", shape=(?, 1560, 17), dtype=float32)
auc in  (?, 1560, 18)
combined (?, 1560, 49)
Feat combined shape (?, 1165)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AUR_input (InputLayer)          (None, 1560, 17)     0                                            
__________________________________________________________________________________________________
AUC_input (InputLayer)          (None, 1560, 18)     0                                            
__________________________________________________________________________________________________
EYE_input (InputLayer)          (None, 1560, 8)      0                                            
__________________________________________________________________________________________________
Pose_input (InputLayer)         (None, 1560, 6)      0        

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


ValueError: ignored